# Use LD Proxy API to find the relevant positions for each of the PRS313 SNPs

In [8]:
import requests
import pandas as pd
import os
import io


# API key
api_key = "ac16be4ad92d"

# Base URL for LDProxy API
base_url = "https://ldlink.nih.gov/LDlinkRest/ldproxy"

population = "ALL"

window = 1000000

r2_threshold = 0.01

# Read the chromosome positions from the text file
PRS313_LD = pd.read_excel("../../../Data/PRS313_with_23andMe.xlsx")


In [9]:
# Create a folder to store the CSV files
output_folder = "../../../Data/ld_variants"
os.makedirs(output_folder, exist_ok=True)

# Iterate over each position
for index, sample in PRS313_LD.iterrows():
    # if sample["in_23andMe"] == True:
    #      continue

    # Extract chromosome and position from the line
    chrom = sample["Chromosome"]
    pos = sample["Positionb"]
    
    # Construct the API request URL
    url = f"{base_url}?var={chrom}:{pos}&pop={population}&r2_d=r2&window={window}&genome_build=grch37&token={api_key}"
    
    # Send the API request
    response = requests.get(url)
    
    # Check if the request was successful
    if response.status_code == 200:
            # Create a StringIO object from the data
            data_io = io.StringIO(response.text)

            # Read the data into a DataFrame using read_csv
            df = pd.read_csv(data_io, sep='\\t')
            
            # # Filter variants with high LD scores (e.g., R2 >= 0.8)
            # high_ld_variants = df[df["R2"].astype(float) >= 0.8]
            
            # Generate a unique filename for the CSV file
            output_file = os.path.join(output_folder, f"{chrom}_{pos}.csv")
            
            # Save the high LD variants to a CSV file
            df.to_csv(output_file, index=False)
            
            print(f"Saved high LD variants for {chrom}:{pos} to {output_file}")
    else:
        print(f"Failed to retrieve data for {chrom}:{pos}. Status code: {response.status_code}")

KeyboardInterrupt: 

In [ ]:
import os
import shutil

# Base folder where the CSV files are currently stored
base_folder = "../../../Data/ld_variants"

# Get a list of all CSV files in the base folder
csv_files = [f for f in os.listdir(base_folder) if f.endswith('.csv')]

# Iterate over each CSV file
for file in csv_files:
    # Extract the chromosome number from the file name
    chrom = file.split('_')[0]
    
    # Create a folder for the current chromosome if it doesn't exist
    chrom_folder = os.path.join(base_folder, f"chr{chrom}")
    os.makedirs(chrom_folder, exist_ok=True)
    
    # Move the file to the corresponding chromosome folder
    source_file = os.path.join(base_folder, file)
    destination_file = os.path.join(chrom_folder, file)
    shutil.move(source_file, destination_file)
    
    print(f"Moved {file} to {chrom_folder}")


Moved 6_152432902.csv to ../../../Data/ld_variants/chr6
Moved 5_79180995.csv to ../../../Data/ld_variants/chr5
Moved 7_25569548.csv to ../../../Data/ld_variants/chr7
Moved 6_152055978.csv to ../../../Data/ld_variants/chr6
Moved 5_44508264.csv to ../../../Data/ld_variants/chr5
Moved 12_14413931.csv to ../../../Data/ld_variants/chr12
Moved 4_106069013.csv to ../../../Data/ld_variants/chr4
Moved 3_55970777.csv to ../../../Data/ld_variants/chr3
Moved 3_59373745.csv to ../../../Data/ld_variants/chr3
Moved 1_88156923.csv to ../../../Data/ld_variants/chr1
Moved 6_16399557.csv to ../../../Data/ld_variants/chr6
Moved 12_115796577.csv to ../../../Data/ld_variants/chr12
Moved 5_131640536.csv to ../../../Data/ld_variants/chr5
Moved 2_218292158.csv to ../../../Data/ld_variants/chr2
Moved 10_38523626.csv to ../../../Data/ld_variants/chr10
Moved 1_120257110.csv to ../../../Data/ld_variants/chr1
Moved 8_106358620.csv to ../../../Data/ld_variants/chr8
Moved 22_46283297.csv to ../../../Data/ld_variants/

# Find overlap b/w 23AndMe and LD


In [ ]:
import os
import pandas as pd
import re
import numpy as np

import collections


directory = "../../../Data/ld_variants"

# Create a folder to store the CSV files
output_folder = "../../../Data/ld_variants"
os.makedirs(output_folder, exist_ok=True)

def extract_coord_alleles(col_name):
    match = re.search(r'chr(\d+)_(\d+)_([ACGT])_([ACGT])$', col_name)
    if match:
        chr_num, position, ref_allele, alt_allele = match.groups()
        return f'chr{chr_num}:{position}', f'({ref_allele}/{alt_allele})'
    return None, None

# Create an empty list to store the matching variants across all chromosomes
matching_variants_all = []

training_data_folder = "../../../Data/Raw_training_data_23andMe_union/"
output_folder = "../../../Data/Data/Filtered_raw_training_data_union/"

# Iterate over each chromosome folder
for chrom_folder in os.listdir(directory):
    if chrom_folder.startswith("chr"):
        chrom = chrom_folder[3:]
        files_folder = os.path.join(directory, chrom_folder)

        # Load the training data for the chromosome
        training_data = pd.read_parquet(f"{training_data_folder}23AndMe_PRS313_merged_chr{chrom}.parquet")

        matching_columns = []
        matching_variants_chrom = []
        not_found_snps_chrom = []

        for filename in os.listdir(files_folder):
            if filename.endswith(".csv"):
                PRS313_SNP_pos = filename.split('.')[0]
                position_column = training_data.columns[training_data.columns.str.contains(PRS313_SNP_pos)]


                try:
                    variants = pd.read_csv(os.path.join(files_folder, filename))

                    for column in training_data.columns:
                        coord, alleles = extract_coord_alleles(column)

                        if coord is not None and alleles is not None:
                            mask = (variants['Coord'] == coord & (variants['Alleles'] == alleles))

                            if mask.any():
                                if column not in matching_columns:
                                    matching_columns.append(column)
                                    matching_variants_chrom.append(variants[mask])
                except KeyError:
                    error_position = filename.split('.')[0]
                    not_found_snps_chrom.append(error_position)
                    print(f"SNP {error_position} not found in dbSNP and cannot be proxied using LDProxy")

                    counter_error_added = 0
                    for col in training_data.columns:
                        coord, _ = extract_coord_alleles(col)
                        if coord is not None:
                            col_position = int(coord.split(':')[1])
                            error_bp = int(error_position.split('_')[1])
                            if abs(col_position - error_bp) <= 500000 and col not in matching_columns:
                                counter_error_added += 1
                                
                                coord, alleles = extract_coord_alleles(col)

                                if coord is not None and alleles is not None:
                                    matching_columns.append(col)

                                    empty_structure = pd.DataFrame({
                                        'RS_Number': [np.nan],
                                        'Coord': coord,
                                        'Alleles': alleles,
                                        'MAF': [np.nan],
                                        'Distance': [np.nan],
                                        'Dprime': [np.nan],
                                        'R2': [np.nan],
                                        'Correlated_Alleles': [np.nan],
                                        'FORGEdb': [np.nan],
                                        'RegulomeDB': [np.nan],
                                        'Function': [np.nan]
                                    })
                                    matching_variants_chrom.append(empty_structure)
    
                    print(f"Added {counter_error_added} columns to matching_columns for missing data position: {error_position}")

        matching_data = training_data[matching_columns]

        os.makedirs(output_folder, exist_ok=True)
        save_path = f"{output_folder}/23AndMe_PRS313_merged_chr{chrom}_matching.parquet"
        matching_data.to_parquet(save_path)

        matching_variants_chrom_df = pd.concat(matching_variants_chrom, ignore_index=True)
        matching_variants_all.append(matching_variants_chrom_df)

        print(f"Saved to file {save_path}")
        print(f"Found {len(matching_columns)} matching columns")
        print(f"Found {len(matching_variants_chrom_df)} matching variants")
        if (len(matching_columns) != len(matching_variants_chrom_df)):
            duplicates = [item for item, count in collections.Counter(matching_variants_chrom_df.Coord).items() if count > 1]
            matching_variants_chrom_df.loc[matching_variants_chrom_df.Coord.isin(duplicates)]
            print("SOMETHIGN HAS GONE WRONG")

matching_variants_all_df = pd.concat(matching_variants_all, ignore_index=True)
matching_variants_all_df.to_csv(f"{output_folder}/23AndMe_matching_variants.csv", index=False)


KeyboardInterrupt: 

In [48]:
import os
import pandas as pd
import re
import numpy as np

directory = "../../../Data/ld_variants"

# Create a folder to store the CSV files
output_folder = "../../../Data/ld_variants"
os.makedirs(output_folder, exist_ok=True)

# # Regex used to parse the 23andMe data. Can't use this version because it looks for locations where there are more than one nucleotide in the ref/alt alleles
# def extract_coord_alleles(col_name):
#     match = re.search(r'chr(\d+)_(\d+)_([ACGT,]+)_([ACGT,]+)', col_name)
#     if match:
#         chr_num, position, ref_allele, alt_allele = match.groups()
#         return f'chr{chr_num}:{position}', f'({ref_allele}/{alt_allele})'
#     return None, None

# Only gets the 23andMe data with a single nucleotide for ref/alt 
def extract_coord_alleles(col_name):
    match = re.search(r'chr(\d+)_(\d+)_([ACGT])_([ACGT])$', col_name)
    if match:
        chr_num, position, ref_allele, alt_allele = match.groups()
        return f'chr{chr_num}:{position}', f'({ref_allele}/{alt_allele})'
    return None, None

# Create an empty list to store the matching variants across all chromosomes
matching_columns_all = []

training_data_folder = "../../../Data/Raw_training_data_23andMe_union/"
output_folder = "../../../Data/Filtered_raw_training_data_union/"
no_ld_positions = 0

# Iterate over each chromosome folder
for chrom_folder in os.listdir(directory):
    if chrom_folder.startswith("chr"):
        chrom = chrom_folder[3:]
        files_folder = os.path.join(directory, chrom_folder)

        # Load the training data for the chromosome
        training_data = pd.read_parquet(f"{training_data_folder}23AndMe_PRS313_merged_chr{chrom}.parquet")

        # Extract positions from training_data.columns
        training_coords = training_data.columns.to_series().apply(lambda col: extract_coord_alleles(col)[0])
        training_coords.name = "Coord"

        matching_columns = []
        not_found_snps_chrom = []

        for filename in os.listdir(files_folder):
            if filename.endswith(".csv"):
                
                # Use to add PRS313 SNPs to the training data, since the regex in extract_coord_alleles will no longer match multiallelic/indels for mutations
                position = filename.split('.')[0]
                position_column = training_data.columns[training_data.columns.str.contains(position)]
                if (len(position_column) > 1):
                    raise Exception("More than 1 PRS313 SNP")
                # Add position_columnt o matching_columns if it isn't in there
                if (position_column not in matching_columns):
                    matching_columns.append(position_column[0])

                try:
                    variants = pd.read_csv(os.path.join(files_folder, filename))

                    # Find matching columns based on coordinates
                    matched_columns = training_coords[training_coords.isin(variants['Coord'].values)].index.tolist()
                    matching_columns.extend([str(col) for col in matched_columns if col not in matching_columns])

                except KeyError:
                    error_position = filename.split('.')[0]
                    not_found_snps_chrom.append(error_position)
                    no_ld_positions+=1
                    # Find close columns using the training_coords
                    error_bp = int(error_position.split('_')[1])
                    close_coords = training_coords[training_coords.apply(lambda coord: coord is not None and abs(int(coord.split(':')[1]) - error_bp) <= 500000)]
                    close_cols = close_coords.index.tolist()
                    matching_columns.extend([str(col) for col in close_cols if col not in matching_columns])

                    print(f"Added {len(close_cols)} columns to matching_columns for missing data position: {error_position}")

        # Append matching columns for the current chromosome to the overall list
        matching_columns_all.extend(matching_columns)

        matching_data = training_data[matching_columns]

        os.makedirs(output_folder, exist_ok=True)
        save_path = f"{output_folder}23AndMe_PRS313_merged_chr{chrom}_matching.parquet"
        matching_data.to_parquet(save_path)

        print(f"Saved to file {save_path}")
        print(f"Found {len(matching_columns)} matching columns for chromosome {chrom}")

# Save the list of all matching columns to a CSV file
matching_columns_df = pd.DataFrame({'matching_columns': matching_columns_all})
matching_columns_df.to_csv(f"{output_folder}/matching_columns_all.csv", index=False)

print("All processing done and matching columns saved to CSV.")


KeyboardInterrupt: 

In [29]:
all_pos = "../../../Data/Data/Filtered_raw_training_data_union/matching_columns_all.csv"
missing_pos = "../../../Data/Filtered_raw_training_data_union/matching_columns_all.csv"

all_pos = pd.read_csv(all_pos)
missing_pos = pd.read_csv(missing_pos)
missing_columns = set(missing_pos["matching_columns"])
all_columns = set(all_pos["matching_columns"])
difference = all_columns.difference(missing_columns)
difference

{'chr10_123340431_GC_G_PRS313_Unknown',
 'chr10_95292187_CAA_C_PRS313_Unknown',
 'chr14_91751788_TC_T_PRS313_Unknown',
 'chr16_4008542_CAAAAA_C_PRS313_Unknown',
 'chr18_24518050_AT_A_PRS313_Unknown',
 'chr1_110198129_CAAA_C_PRS313_Unknown',
 'chr1_168171052_CA_C_PRS313_Unknown',
 'chr1_46670206_TC_T_PRS313_Unknown',
 'chr1_51467096_CT_C_PRS313_Unknown',
 'chr22_40904707_CT_C_PRS313_Unknown',
 'chr2_217955896_GA_G_PRS313_Unknown',
 'chr3_141112859_CTT_C_PRS313_Unknown',
 'chr4_92594859_TTCTTTC_T_PRS313_Unknown',
 'chr5_77155397_GT_G_PRS313_Unknown',
 'chr6_152022664_CAAAAAAA_C_PRS313_Unknown',
 'chr6_20537845_CA_C_PRS313_Unknown',
 'chr6_82263549_AAT_A_PRS313_Unknown',
 'chr6_85912194_CAA_C_PRS313_Unknown',
 'chr7_139943702_CT_C_PRS313_Unknown',
 'chr8_17787610_CT_C_PRS313_Unknown',
 'chr9_110303808_TAA_T_PRS313_Unknown',
 'chr9_21964882_CAAAA_C_PRS313_Unknown'}

In [34]:
missing_pos.loc[missing_pos["matching_columns"].str.contains("PRS313")][50:100]

,matching_columns
2793,chr4_151218296_CATATTT_C_PRS313_Unknown
2913,chr4_89240476_G_A_PRS313_Unknown
2941,chr4_143467195_C_T_PRS313_Unknown
3376,"chr4_187503758_A_G,T_PRS313_Unknown"
3626,"chr4_126752992_A_AAT,AATAT_PRS313_Unknown"
3874,"chr4_84370124_TAA_TA,T_PRS313_Unknown"
4085,chr3_55970777_A_AT_PRS313_Unknown
4112,chr3_59373745_C_T_PRS313_Unknown
4335,chr3_99403877_G_A_PRS313_Unknown
4345,chr3_172285237_G_A_PRS313_Known


# Optional - Create Final Training Data with +/- 500K BP Window

In [ ]:
import os
import pandas as pd
import re

window_size = 250000
pattern = re.compile(r"chr\d+_(\d+)_")

# Create an empty list to store the matching variants across all chromosomes
matching_variants_all = []

for chrom in range(1, 23):
    # Load the training data for the chromosome
    training_data = pd.read_parquet(f"../../../Data/Raw_training_data/23AndMe_PRS313_merged_chr{chrom}.parquet")
    
    # Get all columns with "PRS313" in the name
    prs313_unknown_columns = [col for col in training_data.columns if "PRS313_Unknown" in col]
    prs313_unknown_positions = [int(pattern.search(col).group(1)) for col in prs313_unknown_columns]
    prs313_unknown_positions_set = set(prs313_unknown_positions)
    
    # Get all columns in training_data that contain a number within +/- 500k of the PRS313_Unknown position
    filtered_columns = [col for col in training_data.columns if any(abs(int(pattern.search(col).group(1)) - pos) <= window_size for pos in prs313_unknown_positions_set)]
    
    training_data_filtered = training_data[filtered_columns]

    print(len(filtered_columns))

    # Save the filtered training data for the chromosome
    # training_data_filtered.to_parquet(f"../../Data/500k_window_filtered_data/23AndMe_PRS313_merged_chr{chrom}_filtered.parquet")

FileNotFoundError: [Errno 2] No such file or directory: '../../Data/Raw_training_data/23AndMe_PRS313_merged_chr1.parquet'

In [ ]:
chrom = 1

# training_data_window = pd.read_parquet(f"../../../Data/500k_window_filtered_data/23AndMe_PRS313_merged_chr{chrom}_filtered.parquet")
# print(training_data_window.shape)

training_data_ld_proxy = pd.read_parquet(f"../../../Data/Data/Filtered_raw_training_data_union/23AndMe_PRS313_merged_chr1_matching.parquet/23AndMe_PRS313_merged_chr{chrom}_matching.parquet")
print(training_data_ld_proxy.shape)

(2504, 2039)
